<a href="https://colab.research.google.com/github/Lornng/FineTune/blob/main/Evaluation_of_Llama2_7B_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Required Libraries

In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Log in to Hugging Face

In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Load & Quantize the model

In [38]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

# model_id = "chloecchng/llama2_merged"
model_id ='Lornng/Llama2-7b-merged-qlora-cpgQA-text-gen-final'
# model_id = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
#model = LlamaForQuestionAnswering.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [39]:
from transformers import pipeline

pipe = pipeline("question-answering", model = model, tokenizer=tokenizer)
pipe(question="What is my name?", context="My name is Wolfgang and asjdhwhawdawdawfawfawfawf. jaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaahdawiyyyyyyyyygdawdawdawdaajkawhuiawyuidahfhjavsjvvbajjwihioqwyu bsyfk qiw qwfuigaiu aiiugwiu giqwhfgagf yukwt 87g qiu gfhugafjgqwg f iqgjhavfh gqfiy iqhgsdhfg lrgoi wyogsuidgouwyaoy zdgfuyq9hfiudsg fuiqo ogusagf iugah oah syfo yq3gasgf ahlhoi a yoryq h jlkgaklsgf agfkj iu3tyu igaluigsfi uwtigsdufg iy olha4uy 4ihulh di uyf y498 ylahd uifhuaih 4iuha lduh fijkgh euy4uhuahu hiualeh iu yuialuieyl48ahuuliah  uliiuaehuiyiu4yiuyaiudhl ukahiuly4iuyiualh diu halui4yiuahul aiyr aiuydu yair  ghgdaihla k hla yioar hiuah ikayiuyaiuylouy aiuhduih ar")

The model 'LlamaForCausalLM' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltForQuestionAnswering', 'Longforme

KeyError: ignored

# Load and preprocessing the dataset

In [48]:
from datasets import load_dataset

# Load test dataset
test_data = load_dataset("Lornng/cpgQA-textcol-splitted", split='test')

# Preprocessing test and validation dataset
def group_test_data(dataset):

    grouped = []
    ref_answers = []

    for row in dataset:
      #input = "Question: " + row['question'] + " Context: " + row['context'] + " Answer: "
      input = "Question: " + row['QUESTION'] + " Context: " + row['CONTEXT']

      grouped.append(input)

      ref_answer = row['ANSWER']
      ref_answers.append(ref_answer)

    return grouped, ref_answers

# Test dataset
output_column, reference = group_test_data(test_data)
test_data = test_data.add_column('Outputs', output_column)

In [49]:
from transformers import pipeline

pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    # stopping_criteria=stopping_criteria,
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=100,  # max number of tokens to generate in the output
    repetition_penalty=1.1 # without this output begins repeating
)

outputs = []

for out in pipe(test_data['Outputs']):
  outputs.append(out)

In [50]:
outputs

[[{'generated_text': ' Answer:Availability of accurate and timely confirmatory testing (e.g., gas chromatography-mass spectrometry  [GCMS]) is critical due to the false positive and negative rates associated with UDTs.[53]  Explanation:According to the VA/DoD Clinical Practice Guideline for Management of Substance Use Disorders,  UDTs, used in the appropriate way, help to address safety,'}],
 [{'generated_text': ' Answer:45 mg SR (15 mg x 3) Q8h for day 2, 30 mg SR (15 mg x 2) Q8h for day 3, 15 mg SR Q8h for day 4, 15 mg SR Q12h for day 5-7, 15 mg SR QHS for day 8-11. Stop rapid tapering after day'}],
 [{'generated_text': ' Answer: Establish treatment goals before starting opioid therapy and a plan if therapy is discontinued. Only continue opioid if there is clinically meaningful improvement in pain and function. Discuss risks, benefits and responsibilities for managing therapy before starting and during treatment. 108 Improper Use/Abuse of Prescription Medications and Illegal Drug Use

In [51]:
import ast
import nltk

import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install tabulate

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [52]:
from tabulate import tabulate

# Create a list to store the results in a table
table_data = []

# Calculate F1 and exact match scores
exact_match_count = 0
f1_score_total = 0

for i, (predicted_dict_list, actual) in enumerate(zip(outputs, reference)):
    # Initialize an empty list to store generated text from dictionaries
    predicted_texts = []

    # Iterate through the list of dictionaries and extract 'generated_text' from each
    for predicted_dict in predicted_dict_list:
        predicted_texts.append(predicted_dict.get('generated_text', ''))

    # Convert the list of generated texts to a single string
    predicted = ' '.join(predicted_texts)

    # Tokenize the predicted and actual answers
    predicted_tokens = nltk.word_tokenize(predicted)
    actual_tokens = nltk.word_tokenize(actual)

    # Calculate exact match
    if predicted == actual:
        exact_match_count += 1

    # Calculate precision and recall
    if len(predicted_tokens) > 0 and len(actual_tokens) > 0:
        common_tokens = set(predicted_tokens) & set(actual_tokens)
        precision = len(common_tokens) / len(predicted_tokens)
        recall = len(common_tokens) / len(actual_tokens)

        if precision + recall > 0:
            f1 = (2 * precision * recall) / (precision + recall)
            f1_score_total += f1

    # Append the results to the table
    table_data.append([i, predicted, actual, precision, recall, f1])

# Print the table
headers = ["Index", "Predicted Answer", "Actual Answer", "Precision", "Recall", "F1 Score"]
# print(tabulate(table_data, headers=headers, tablefmt="pretty"))

# Calculate scores
exact_match_score = exact_match_count / len(reference)
average_f1_score = f1_score_total / len(reference) if len(reference) > 0 else 0

print("Exact Match:", exact_match_score)
print("Average F1 Score:", average_f1_score)


Exact Match: 0.0
Average F1 Score: 0.19460218028667142


In [ ]:
replace_list = ['\\', '\n', '`', '#', 'Comment']

formatted_prediction = []

for row in prediction:
  for replace in replace_list:
    row = row.replace(replace, '')

  formatted_prediction.append(row)




In [ ]:
formatted_prediction

['3 1/2 c. (8 oz.) large shells (uncooked)',
 '3 hours',
 '4 (1 oz.) slices Canadian bacon',
 '1/2 c.',
 '15 minutes',
 '3 eggs, 1 c. oil, 1 1/2 c. honey, 2 tsp. vanilla, 2 c. shredded zucchini, 1 c. unsweetened crushed pineapple, 1 c. raisins, 2 c. whole wheat flour, 3/4 c. unbleached flour, 2 tsp. soda, 1 tsp. salt, ',
 '1 (14 oz.) can evaporated skim milk, 1/2 c. vinegar, 1 envelope (about 1 1/2 oz.) onion soup mix, 1/3 c. catsup, 1/2 tsp. Worcestershire sauce, 1/4 tsp. red pepper sauce',
 '1. Mix all ingredients except the ketchup.2. Put mixture in a large iron skillet.3. Spoon ketchup over top.4. Bake, uncovered, in a 350° oven for 1 hour and 15 minutes.',
 '2 lb. ground meat',
 '3 layers',
 '1/3 cup of oil',
 '8 minutes',
 '8 minutes',
 '1. Cook all ingredients in 1 1/2-quart saucepan over medium heat for 4 to 5 minutes, stirring vigorously, until mixture forms a ball.2. Remove dough from saucepan and let stand on counter 5 minutes.3. Knead dough about 30 seconds or until smooth 

In [ ]:
df = pd.DataFrame(formatted_prediction)
df.to_csv('prediction.csv')

In [ ]:
reference

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score

binarizer = MultiLabelBinarizer()

# This should be your original approach
#binarizer.fit(your actual true output consisting of all labels)

# In this case, I am considering only the given labels.
binarizer.fit(reference)

f1_score(binarizer.transform(reference),
         binarizer.transform(formatted_prediction),
         average='samples')

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['+', '=', 'Q', 'V', 'é'] will be ignored
  warnings.warn(


0.7141221837959175